# Reinforce 구현

In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Hyperparameters
learning_rate = 0.0002
gamma = 0.98

In [2]:
def main():
    env = gym(make="CartPole-v1", render_mode="human")
    pi = Policy()
    score = 0.0
    print_interval = 20

    for n_epi in range(10000):
        done = False
        s = env.reset()
        env.render()

        while not done:
            prob = pi(torch.from_numpy(s).float())
            m = Categorical(prob) # policy를 기반으로 분포를 생성
            a = m.sample() # 분포로부터 action을 샘플링
            s_prime, r, done, info = env.step(a.item()) # a.item() : tensor에서 숫자를 꺼내는 함수
            pi.put_data((r, prob[a])) # reward와 해당 action의 확률을 저장
            s = s_prime
            score += r
        
        pi.train_net() # episode가 끝날 때마다 policy를 업데이트 G_t
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode : {}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    env.close() 



In [3]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []

        self.fc1 = nn.Linear(4, 128) # 4개의 state를 받아서 128개의 노드로 보낸다. (이 환경의 state는 4개)
        self.fc2 = nn.Linear(128, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0) # softmax는 확률분포를 출력한다.
        return x
    
    def put_data(self, item):
        self.data.append(item)
    
    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R 
            loss.backward()
        self.optimizer.step()
        self.data = []

AttributeError: module 'torch.nn' has no attribute 'module'